In [ ]:
rn = n.random.rand(200,1)

In [ ]:
# bitsler study
import pandas as p
import numpy as n
from qoreliquid import normalizeme
from qoreliquid import sigmoidme
%pylab inline

def calc(rn, ini=100, risk=1, payout=3, winningEvent=-50, losingEvent=25, batch=False):
    ini = float(ini)
    risk = round(risk, 8)
    df = p.DataFrame(rn)    
    chance = float(99) / payout
    wchance = (chance / 100)
    lchance = (1 - (chance / 100))
    winners = df[df[0] < wchance].index
    losers  = df[df[0] > lchance].index
    #print ini
    df.ix[0, 'capital'] = ini
    for i in winners:
        df.ix[i, 'b'] = 1
    for i in losers:
        df.ix[i, 'a'] = 1
    for i in df.index:
        df.ix[i, 'risk']  = risk # df.ix[i, 'capital'] * risk / 100
    df['payout'] = payout
    df['chance'] = chance
    df['wchance'] = wchance
    df['lchance'] = lchance
    df['winningEvent'] = winningEvent
    df['losingEvent'] = losingEvent
    df = df.fillna(0)
    df['win'] = n.array((df['b']), n.int32)
    df['lose'] = n.array(n.not_equal(df['b'], 1), n.int32)

    #for i in df.index:
    #    try: df.ix[i, 'deposit'] = df.ix[i, 'capital'] * 1 / 100
    #    except: ''
    df['xrisk'] = risk #/ 100
    for i in df.index:
        df.ix[i, 'iCapital'] = df.ix[i, 'capital']
        #df.ix[i, 'xrisk']    = (df.ix[i, 'capital'] * risk / 100)
        #if df.ix[i, 'capital'] > 0:
        #    df.ix[i, 'xrisk']    = risk #/ 100
        if df.ix[i, 'capital'] <= 0:
            df.ix[i, 'xrisk'] = 0 #/ 100
        xpayoutWin  = (((df.ix[i, 'xrisk'] * df.ix[i, 'payout']) - df.ix[i, 'xrisk']) * df.ix[i, 'win'])
        xpayoutLose = (df.ix[i, 'xrisk'] * df.ix[i, 'lose'])
        if df.ix[i, 'capital'] > 0:
            #0.011236*(100+25.0)/100
            winPcent = (float(df.ix[i, 'winningEvent'] * df.ix[i, 'win'])/100)
            losePcent = (float(df.ix[i, 'losingEvent'] * df.ix[i, 'lose'])/100)
            #df.ix[i+1, 'xrisk'] = df.ix[i, 'xrisk'] * (1+float(df.ix[i, 'winningEvent'] * df.ix[i, 'win'])/100)
            #df.ix[i+1, 'xrisk'] = df.ix[i, 'xrisk'] * (1+float(df.ix[i, 'losingEvent'] * df.ix[i, 'lose'])/100)
            df.ix[i+1, 'xrisk'] = df.ix[i, 'xrisk'] * (1 + winPcent + losePcent)
        df.ix[i, 'xpayout'] =  round(xpayoutWin - xpayoutLose,8) #* df.ix[i, 'xrisk']
        df.ix[i, 'capital'] = df.ix[i, 'xpayout'] + df.ix[i, 'capital']
        #df.ix[i, 'capital'] += df.ix[i, 'xpayout']
        #print df.ix[i, 'capital']
        try:
            if i < len(df)-1:
                df.ix[i+1, 'capital'] = df.ix[i, 'capital']
                #df.ix[i+1, 'risk'] = df.ix[i, 'capital'] * risk / 100
            ''
        except Exception as e:
            print e
    #print df
    #for i in df.index:
    #    df.ix[i, 'risk'] = df.ix[i, 'capital'] * risk / 100
    #df[0].plot()
    df['xpayoutN'] = normalizeme(df['xpayout'])
    df['xpayoutN'] = sigmoidme(df['xpayoutN'])

    # compute the sharpe ratio
    lin = len(df.index)-1
    df.ix[1:lin,'returns'] = 100 - ( df.ix[0:lin-1,'capital'].get_values() / df.ix[1:lin,'capital'].get_values() * 100 )

    #ca = normalizeme(df['capital'])
    ca = df['capital'] / df.ix[0,'capital']
    #ca.plot()
    #print ca
    
    sharpe = n.mean(ca) / n.std(df['returns'])
    #print 'sharpe: %s' % sharpe

    df = df.fillna(0)
    if batch == False:
        #df.ix[:, ['xpayoutN']].plot()
        #plt.show()
        #df.ix[:, ['xpayout']].plot()
        #plt.show()
        df.ix[:, ['capital']].plot()
        plt.show()incom
        with p.option_context('display.max_rows', 15, 'display.max_columns', 4000, 'display.width', 1000000): print df

    return {
        'balance': df.ix[df.tail(1).index[0], 'capital'], 
        'balancePcnt': df.ix[df.tail(1).index[0], 'capital']/ df.ix[df.head(1).index[0], 'iCapital'] * 100 - 100, 
        'sharpe':  sharpe,
        'winningEvent': winningEvent,
        'losingEvent': losingEvent,
    }

#rn2 = n.random.rand(2000,1)a
#calc(rn,10000)

rn3 = n.random.rand(50,1)
#calc(rn3,1.04e-6, 2, 3)
ic = 300
#ic = 0.88e-6
res = calc(rn3,ic, float(ic)/89, payout=3, winningEvent=-50, losingEvent=25)
print res

In [ ]:
# test
risk = 34
lns = []
rn3 = n.random.rand(100,1)
#"""
for i in range(20):
    ic = 0.88e-6
    rWinningEvent = n.random.randint(-99,50)
    rLosingEvent = n.random.randint(-99,50)
    res = calc(rn3,ic, float(ic)/risk, 3, winningEvent=rWinningEvent, losingEvent=rLosingEvent, batch=True)
    #print res
    lns.append(res)
#"""
dflns = p.DataFrame(lns)
dflns['a'] = dflns['balancePcnt'] * dflns['sharpe']
rdflns = (dflns[dflns['sharpe'] > 0].sort_values(by='a', ascending=False))
rdflns = dflns[dflns['balancePcnt'] > 0]

print rdflns
print '====='


for k in rdflns.index:
    # test2
    lns = []
    #risk = 144
    #"""
    for j in range(20):
        i = rdflns.index[k]
        rn3 = n.random.rand(200,1)
        ic = 698e-6
        res = calc(rn3,ic, float(ic)/risk, 3, winningEvent=rdflns.ix[i, 'winningEvent'], losingEvent=rdflns.ix[i, 'losingEvent'], batch=True)
        print res
        lns.append(res)
    #"""
    dflns2 = p.DataFrame(lns)
    dflns2['a'] = dflns2['balancePcnt'] / dflns2['sharpe'] * (dflns2['sharpe'] / n.abs(dflns2['sharpe']))
    #dflns2 = dflns2[dflns2['sharpe'] > 0]
    #dflns2 = dflns2[dflns2['a'] > 0]
    dflns2 = dflns2.sort_values(by='a', ascending=False)
    print dflns2#.sort_values(by='a', ascending=False)

In [ ]:
i = dflns2.index[0]
ic = 698e-6
rn3 = n.random.rand(100,1)
print calc(rn3,ic, float(ic)/risk, 3, winningEvent=dflns2.ix[i, 'winningEvent'], losingEvent=dflns2.ix[i, 'losingEvent'], batch=False)

In [ ]:
dflnsN = (dflns)
#dflnsN = normalizeme(dflnsN)
#dflnsN = sigmoidme(dflnsN)
scatter(dflnsN['losingEvent'] / dflnsN['winningEvent'], dflnsN['sharpe'])
plt.show()
dflnsN

In [ ]:
dflnsN = (dflns)
#dflnsN = normalizeme(dflnsN)
#dflnsN = sigmoidme(dflnsN)
for i in dflnsN.sort_values(by='sharpe', ascending=False).head(5).index:
    #print dflnsN.ix[i, :]
    ic = 300
    print calc(rn3,ic, ic/89, 3, winningEvent=dflnsN.ix[i, 'winningEvent'], losingEvent=dflnsN.ix[i, 'losingEvent'], batch=False)
#.plot() #['sharpe']

In [ ]:
#ini = [100, 0.00000300, 0.0000300]
ini = []
#for i in range(1,4):
#    ini.append(1.0 / n.powini = []
for i in list(n.arange(4, 0, -1)): ini.append(n.power(10,i))
for i in range(0,9):               ini.append(1.0 / n.power(10,i))
print ini
for i in range(len(ini)): calc(rn,ini[i])